### INTRODUCTION

## 

### BUSINESS UNDERSTANDING

##

### PROBLEM STATEMENT

##


### OBJECTIVES

##

## DATA UNDERSTANDING

### We are going to be using the data Twitter sentiments  about google and apple products  from CrowdFlower. This is the  [Link] to the data. The csv file contains  the following  variables

- `Tweet_text`: Tweet content
- `semotion_in_tweet_is_directed_at`: The type of device from which the twitter sentimnent was sent
- `emotion`: Emotion of the tweet ,the emotion is classifies as Negative, Positive ,no emotion  or a state where the person could not classify the emotion in the tween , labeled as "I can't tell"



## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## Loading Data

In [9]:
import pandas as pd
data = pd.read_csv('data\judge-1377884607_tweet_product_company.csv',encoding='ISO-8859-1')
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
